In [2]:
# importing relevant libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.linear_model import LinearRegression

In [4]:
# importing the cleaned dataset into jupyter notebooks

prem_transfers = pd.read_excel('premier_league_transfers.xlsx',sheet_name='cleaned')

prem_transfers.head()

,year,player_name,age,new_position,transfer_from,transfer_to,transfer_period,transfer_fee
0,1992,Tommy Wright,26.0,Attacker,Leicester,Middlesbrough FC,Summer,910000.0
1,1992,Jonathan Gittens,28.0,Defender,Southampton,Middlesbrough FC,Summer,250000.0
2,1992,Mark Robins,22.0,Attacker,Man Utd,Norwich City,Summer,1000000.0
3,1992,Gary Megson,33.0,Midfielder,Man City,Norwich City,Summer,0.0
4,1992,Mark Peters,19.0,Defender,Man City,Norwich City,Summer,0.0


## Creating dummy variables for categorical columns

Dummy variables help us convert categorical data into numerical data which are more suitable for a regression model. In the dataset, I would have to convert both the new_position and transfer_period columns from a categorical to a numerical format.

In [13]:
#creating a copy of the original dataset so I can input the dummy variables
pl_transfers = prem_transfers.copy()

#setting summer transfers as 0 and winter transfers as 1
pl_transfers['transfer_period'] = pl_transfers['transfer_period'].map({'Summer':0,'Winter':1})

#creating dummy variables for new_position
pos_dummies = pd.get_dummies(pl_transfers['new_position'],drop_first=True)# we drop the first entry (Attacker in our case) to prevent multicollinearity 

#Adding the new position dummy variables into the dataset
pl_transfers = pd.concat([pl_transfers,pos_dummies], axis=1)

pl_transfers

,year,player_name,age,new_position,transfer_from,transfer_to,transfer_period,transfer_fee,Defender,Goalkeeper,Midfielder
0,1992,Tommy Wright,26.0,Attacker,Leicester,Middlesbrough FC,0,910000.0,0,0,0
1,1992,Jonathan Gittens,28.0,Defender,Southampton,Middlesbrough FC,0,250000.0,1,0,0
2,1992,Mark Robins,22.0,Attacker,Man Utd,Norwich City,0,1000000.0,0,0,0
3,1992,Gary Megson,33.0,Midfielder,Man City,Norwich City,0,0.0,0,0,1
4,1992,Mark Peters,19.0,Defender,Man City,Norwich City,0,0.0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
4142,2022,João Gomes,21.0,Midfielder,Flamengo,Wolverhampton Wanderers,1,18700000.0,0,0,1
4143,2022,Mario Lemina,29.0,Midfielder,OGC Nice,Wolverhampton Wanderers,1,11000000.0,0,0,1
4144,2022,Pablo Sarabia,30.0,Attacker,Paris SG,Wolverhampton Wanderers,1,5000000.0,0,0,0
4145,2022,Craig Dawson,32.0,Defender,West Ham,Wolverhampton Wanderers,1,3750000.0,1,0,0


## Building the regression model

In [14]:
#importing the statsmodels library which will be used for our regression
import statsmodels.api as sm 

#importing the Standard scaler library so we can standardize our predictors (independent variabes).
from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler()

#declaring the dependent(y) and independent(X) variables
X = pl_transfers[['year','age','transfer_period','Defender','Goalkeeper','Midfielder']]
y = pl_transfers['transfer_fee']

#standardizing the independent variables
X_scaled = scaler.fit_transform(X)

#Converting the scaled data back into a Dataframe with the original column names
X_scaled = pd.DataFrame(X_scaled, columns = X.columns)

#Adding a constant to the model
x = sm.add_constant(X_scaled)

#fitting the OLS model and getting a summary of the model
model = sm.OLS(y,x).fit()
model.summary()

C:\Users\Michael\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


MissingDataError: exog contains inf or nans

The error above tells us that there are non numerical values in our dataset. To fix this, I would first have to figure out which column(s) is(are) causing the error.

In [37]:
#identifying the columns with missing values
x.isna().sum()

const              0
year               0
age                0
transfer_period    0
Defender           0
Goalkeeper         0
Midfielder         0
dtype: int64

From the output above, we see that there are two missing values in the age column.

In [19]:
#identifying the exact rows with missing values in our age column
pl_transfers[pl_transfers['age'].isna()]

,year,player_name,age,new_position,transfer_from,transfer_to,transfer_period,transfer_fee,Defender,Goalkeeper,Midfielder
358,1996,Scott Oakes,NaN,Attacker,Luton,Sheffield Wednesday,0,640000.0,0,0,0
3649,2018,Emiliano Sala,NaN,Attacker,FC Nantes,Cardiff City,1,17000000.0,0,0,0


A quick google search is enough to tell us the ages of the players above when these transfers were made. I can now replace the NaN values with the correct ages.

In [20]:
#assigning the right ages to the players with missing age values
pl_transfers.loc[358,'age'] = 24
pl_transfers.loc[3649,'age'] = 28

#confirming that the changes made have been effected
pl_transfers[pl_transfers['age'].isna()]

,year,player_name,age,new_position,transfer_from,transfer_to,transfer_period,transfer_fee,Defender,Goalkeeper,Midfielder


In [47]:
#centering the year and age variables
#this is done so we can predict what the transfer fee is when the year and age variables are at the mean and not when they are 0.
pl_transfers['year_centered'] = pl_transfers['year']-pl_transfers['year'].mean()
pl_transfers['age_centered'] = pl_transfers['age']-pl_transfers['age'].mean()

#declaring the dependent(y) and independent(X) variables
C = pl_transfers[['year_centered','age_centered','transfer_period','Defender','Goalkeeper','Midfielder']]
d = pl_transfers['transfer_fee']

#Adding a constant to the model
c = sm.add_constant(C)

#fitting the OLS model and getting a summary of the model
model = sm.OLS(d,c).fit()
model.summary()

C:\Users\Michael\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           transfer_fee   R-squared:                       0.184
Model:                            OLS   Adj. R-squared:                  0.183
Method:                 Least Squares   F-statistic:                     156.0
Date:                Wed, 20 Nov 2024   Prob (F-statistic):          4.38e-179
Time:                        04:32:03   Log-Likelihood:                -72707.
No. Observations:                4147   AIC:                         1.454e+05
Df Residuals:                    4140   BIC:                         1.455e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            8.299e+06   2.99e+05     27.800      0.000    7.71e+06    8.88e+06
year_centered    5.078e+05   1.87e+04     27.214      0.000    4.71e+05    5.44e+05
age_centered    -1.844e+05   3.54e+04     -5.211      0.000   -2.54e+05   -1.15e+05
transfer_period  -8.41e+05   3.65e+05     -2.303      0.021   -1.56e+06   -1.25e+05
Defender        -2.675e+06   3.94e+05     -6.785      0.000   -3.45e+06    -1.9e+06
Goalkeeper      -5.319e+06   5.63e+05     -9.444      0.000   -6.42e+06   -4.21e+06
Midfielder      -1.272e+06   4.07e+05     -3.127      0.002   -2.07e+06   -4.75e+05
==============================================================================
Omnibus:                     3235.869   Durbin-Watson:                   1.236
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            86285.233
Skew:                           3.535   Prob(JB):                         0.00
Kurtosis:                      24.199   Cond. No.                         36.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""